In [1]:
import os, sys
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pymilvus import MilvusClient

sys.path.append(os.path.abspath('../'))

from app.config import config

In [2]:
os.environ['NVIDIA_API_KEY'] = config['llm_api_key']

model = ChatNVIDIA(model=config['llm_api_model'])

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name=config['embeddings_model'])

/home/chapolin/aio/langchain-llm-example/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.
/home/chapolin/aio/langchain-llm-example/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
client = MilvusClient("../storage/milvus.db")

In [20]:
# query = 'qual o motivo da suspensao dos servicos eletronicos?'
query = 'onde fica a sede da oab rs?'

In [21]:
res = client.search(
  collection_name="oab_documents",
  data=[embeddings.embed_query(query)],
#   filter="subject == 'history'",
  limit=10,
  output_fields=["text", "subject"],
)

res

data: ["[{'id': 131, 'distance': 0.6112189888954163, 'entity': {'text': '2020. Disponível em: https://www.editorajc.com.br/oab-efetiva-espacos-da-mulher-advogada/.\\nAcesso em: 01 abr. 2021', 'subject': 'file_611c23465ce9f.pdf'}}, {'id': 30, 'distance': 0.6056042313575745, 'entity': {'text': '3. Cadastrar em seus contatos os nossos endereços eletrônicos: csiparecer@oabrs.org.br,\\ncsisolenidade@oabrs.org.br e coordcsi@oabrs.org.br. Por meio deles é que você\\nreceberá as comunicações, diligências e convocações referentes ao presente\\nrequerimento', 'subject': '2_16_5cdf15cd2d8c8.pdf'}}, {'id': 141, 'distance': 0.5951207876205444, 'entity': {'text': 'https://www.oabrs.org.br/noticias/86-anos/26988. Acesso em: 06 abr. 2021', 'subject': 'file_611c23465ce9f.pdf'}}, {'id': 137, 'distance': 0.5945316553115845, 'entity': {'text': 'ORDEM DOS ADVOGADOS DO BRASIL DE GOIÁS. CF Valentina Jungmann apresenta\\nprojeto que estabelece paridade entre homens e mulheres nas eleições da OAB. Disponível e

In [28]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Com base nas informacoes abaixo, responda a pergunta do usuario de maneira formal e sucinta"),
    ("system", res[0][0]['entity']['text']),
    ("system", res[0][1]['entity']['text']),
    ("system", res[0][2]['entity']['text']),
    ("system", res[0][3]['entity']['text']),
    ("system", res[0][4]['entity']['text']),
    ("system", res[0][5]['entity']['text']),
    ("system", res[0][6]['entity']['text']),
    ("system", res[0][7]['entity']['text']),
    # ("system", res[0][8]['entity']['text']),
    ("system", res[0][9]['entity']['text']),
    ("user", "{query}")
])

prompt_template
# res[0][0]['entity']['text']

ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Com base nas informacoes abaixo, responda a pergunta do usuario de maneira formal e sucinta')), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='2020. Disponível em: https://www.editorajc.com.br/oab-efetiva-espacos-da-mulher-advogada/.\nAcesso em: 01 abr. 2021')), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='3. Cadastrar em seus contatos os nossos endereços eletrônicos: csiparecer@oabrs.org.br,\ncsisolenidade@oabrs.org.br e coordcsi@oabrs.org.br. Por meio deles é que você\nreceberá as comunicações, diligências e convocações referentes ao presente\nrequerimento')), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='https://www.oabrs.org.br/noticias/86-anos/26988. Acesso em: 06 abr. 2021')), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], 

In [14]:
parser = StrOutputParser()

In [29]:
chain = prompt_template | model | parser

In [31]:
output = chain.invoke({"query": query})

print(output)

A sede da Ordem dos Advogados do Brasil do Rio Grande do Sul (OAB/RS) fica em Porto Alegre, no endereço: Rua Veterinário João Cabral, 95, Bairro Moinhos de Vento, CEP 90150-120.
